[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_07_clustering-published.ipynb)

# Clustering

En este notebook implementaremos algunos de los conceptos de _clustering_ vistos en clase.



In [ ]:
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5] # para ver los gráficos más grandes

## Probando K-means y DBSCAN

In [ ]:
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler

# crear y normalizar dataset
N = 1500
X, y = datasets.make_blobs(n_samples=N, centers=3, cluster_std=1.0,random_state=2211)
X = StandardScaler().fit_transform(X)

# ejecutar k-means con k=2
algorithm = cluster.MiniBatchKMeans(n_clusters=2)
algorithm.fit(X)
y_pred = algorithm.labels_.astype(int)

# graficar
colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
colors = np.hstack([colors] * 20)
plt.scatter(X[:, 0], X[:, 1], color=colors[y_pred].tolist(), s=10)
plt.show()

**Ejercicio** 
1. Encontrar clusters sobre el mismo dataset utilizando la implementacion de sklearn de [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).
2. Experimentar con los hiperparámetros de ambos modelos.

# Implementaciones de algoritmos

En esta sección implementarán K-Means, Clustering Aglomerativo y GMMs.

## K-Means

Recordemos el algoritmo de KMeans:

```
Hiperparámetros: K

1. Ubicar K centroides al azar.
2. Iterar hasta cumplir con algún criterio de convergencia:

   a. Computar distancias (euclidianas) entre cada punto a cada uno de los centroides.

   b. A cada punto, asignar cluster según cercanía a cada centroide.
   
   c. Mover centroides a la media de cada cluster.

El centroide de un cluster es el vector promedio de las instancias pertenecientes al cluster (vector con promedios de cada atributo).
```

1. Definir la función que inicializa los k centroides al azar: `inicializacion_centroides`.

In [ ]:
def inicializacion_centroides(puntos: np.ndarray, k: int) -> np.ndarray:
    """Inicializa los k centroides eligiendo de manera random k de
      los puntos"""
    #COMPLETARS
    return centroides


2. Completar la función `centroides_cercanos` que calcula, para cada punto, el centroide más cercano que será además donde tendremos a qué cluster pertenece cada uno de los puntos.

In [ ]:
def centroides_cercanos(puntos: np.ndarray, centroides: np.ndarray) -> np.ndarray:
    """ Devuelve un array que contiene el indice al centroide más cercano para cada punto."""
    #COMPLETAR
    return distancias

3. Completar la función `mover_centroides` que mueve a los centroides al punto medio de cada cluster y devuelve los nuevos centroides.

In [ ]:
def mover_centroides(puntos: np.ndarray, cercanos: np.ndarray, k: int) -> np.ndarray:
    """ Mueve los centroides a la media de cada cluster."""
    #COMPLETAR
    return nuevos_centroides

4. Probar la implementación de las funciones

In [ ]:
def kmeans(puntos: np.ndarray, k: int, max_iters: int = 100) -> np.ndarray:
    """ Algoritmo K-means. """
    centroides = inicializacion_centroides(puntos, k)
    for _ in range(max_iters):
        nuevos_clusters = centroides_cercanos(puntos, centroides)
        centroides = mover_centroides(puntos, nuevos_clusters, k)
        
    return centroides, nuevos_clusters

In [ ]:
np.random.seed(42)
puntos = np.random.rand(100, 2)  #100 puntos en dimension 2
k = 3  
centroids, closest = kmeans(puntos, k)

plt.scatter(puntos[:, 0], puntos[:, 1], c=closest, s=50, cmap='viridis')
plt.scatter(centroids[:, 0], centroids[:, 1], color='red', s=200, alpha=0.5)
plt.show()


## Clustering aglomerativo


Ahora compleatremos un algoritmo de clustering jerárquico que utiliza una estrategia _bottom-up_.

### Pseudo-código

```
1. Cada punto es un cluster: C1 = x(1); … ; Cn = x(n) 
2. Calcular una matriz de distancias entre todo par de clusters. 
3. Buscar en la matriz de distancia el par de clusters más similares. 
4. Mergear 2 clusters: A y B según su distancia. 
    a. Borrar las filas de la matriz de distancias correspondientes a los clusters A y B. 
    b. Agregar una nueva fila que contenga la distancia entre A ∪ B y el resto de los clusters. 
5. Repetir los pasos 3 y 4 hasta conseguir la cantidad de clusters deseados (o hasta que haya sólo un cluster). 
```

El objetivo será tener una función:

```python
def aglomerativo(datos, num_clusters=1, criterio_distancia="min")
```

Para ello se propone:

1. Implementar la función `inicializar_distancias(datos)` que toma una matriz de numpy, donde cada fila es una instancia y las columnas son los features y devuelve una matriz de distancias. Las matriz debe ser triangular inferior para calcular las distancias, los valores en y sobre la diagonal deben ser `NaN`.
En este caso usaremos la distancia Euclediana.

In [ ]:
# Tests:
X_0 = np.array([[1], [2], [5], [-3]])
y_0_resp = inicializar_distancias(X_0)
y_0_exp = np.array([[np.nan, np.nan, np.nan, np.nan], [1., np.nan, np.nan, np.nan], [ 4., 3., np.nan, np.nan], [4.,  5., 8., np.nan]])
np.testing.assert_allclose(y_0_resp, y_0_exp, rtol=1e-5)
print("Pasé test 0")


X_1 = np.array([[1,2], [2,3], [3,5]])
y_1_resp = inicializar_distancias(X_1)
y_1_exp = np.array([[np.nan, np.nan, np.nan], [1.41421356, np.nan, np.nan], [3.60555128, 2.23606798, np.nan]])
np.testing.assert_allclose(y_1_resp, y_1_exp, rtol=1e-5)
print("Pasé test 1")


X_2 = np.array([[5.1, 3.5, 1.4, 0.2], [4.9, 3. , 1.4, 0.2], [4.7, 3.2, 1.3, 0.2], [4.6, 3.1, 1.5, 0.2], [5. , 3.6, 1.4, 0.2]])
y_2_resp = inicializar_distancias(X_2)
y_2_exp = np.array([[np.nan, np.nan, np.nan, np.nan, np.nan],  [0.53851648, np.nan, np.nan, np.nan, np.nan],  [0.50990195, 0.3, np.nan, np.nan, np.nan],  [0.64807407, 0.33166248, 0.24494897, np.nan, np.nan],  [0.14142136, 0.60827625, 0.50990195, 0.64807407, np.nan]])
np.testing.assert_allclose(y_2_resp, y_2_exp, rtol=1e-5)
print("Pasé test 2")

2. Definir la función `encontrar minimo(distancias)` que toma una matriz de distancias como la que se generó en el punto anterior y devuelve la fila y la columna de su menor valor.

In [ ]:
# Tests:

d_0 = np.array([[np.nan, np.nan, np.nan, np.nan], [1., np.nan, np.nan, np.nan], [ 4., 3., np.nan, np.nan], [4.,  5., 8., np.nan]])
pos_0_resp = encontrar_minimo(d_0)
pos_0_exp = (1,0)
np.testing.assert_allclose(pos_0_resp, pos_0_exp, rtol=1e-5)
print("Pasé test 0")


d_1 = np.array([[np.nan, np.nan, np.nan], [1.41421356, np.nan, np.nan], [3.60555128, 2.23606798, np.nan]])
pos_1_resp = encontrar_minimo(d_1)
pos_1_exp = (1,0)
np.testing.assert_allclose(pos_1_resp, pos_1_exp, rtol=1e-5)
print("Pasé test 1")


d_2 = np.array([[np.nan, np.nan, np.nan, np.nan, np.nan],  [0.53851648, np.nan, np.nan, np.nan, np.nan],  [0.50990195, 0.3, np.nan, np.nan, np.nan],  [0.64807407, 0.33166248, 0.24494897, np.nan, np.nan],  [0.14142136, 0.60827625, 0.50990195, 0.64807407, np.nan]])
pos_2_resp = encontrar_minimo(d_2)
pos_2_exp = (4,0)
np.testing.assert_allclose(pos_2_resp, pos_2_exp, rtol=1e-5)
print("Pasé test 2")

3. Definir la función `actualizar_distancias(distancias, i, j, criterio="min")` que toma una matriz de distancias y recalcula la distancia de un nuevo cluster que sea la unión del que tiene índice $i$ con el de índice $j$, que saque las columnas $i$ y $j$ y agregue una nueva columna/fila al final con las distancias de este nuevo cluster armado con los demás. Debe al menos tener el criterio de distancias entre clusters "mínimo" u "máximo", opcionalmente se puede completar con "vinculación promedio".

In [ ]:
# Tests:

d_0 = np.array([[np.nan, np.nan, np.nan, np.nan], [1., np.nan, np.nan, np.nan], [ 4., 3., np.nan, np.nan], [4.,  5., 8., np.nan]])
pos_0_resp = actualizar_distancias(d_1,1,0)
pos_0_exp = np.array([[np.nan, np.nan], [2.23606798, np.nan]])
np.testing.assert_allclose(pos_0_resp, pos_0_exp, rtol=1e-5)
print("Pasé test 0")


4. Revisar el código de `aglomerativo` y correrlo para el ejemplo

    ```python
        X = np.array([[i] for i in [2, 8, 0, 4, 1, 9, 9, 0]])

        cl, u = aglomerativo(X, criterio="max")

        fig = plt.figure(figsize=(10, 6),facecolor='white')
        dn = dendrogram(u)
    ```

    Verificar que obtienen un resultado equivalente al mostrado como ejemplo de [esta página](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage).

In [ ]:
def aglomerativo(datos, num_clusters=1, criterio_distancia="min"):
    distancias = inicializar_distancias(datos)

    uniones = []
    clusters = [[idx] for idx,_ in enumerate(datos)]
    nombres_de_clusters = [idx for idx,_ in enumerate(datos)]
    prox_cluster = len(nombres_de_clusters) 

    while distancias.shape[0] > num_clusters:
        # Buscar el par de clusters más similares
        i, j = encontrar_minimo(distancias)
        d = distancias[i][j]
        
        # Fusionar los clusters A y B
        nuevo_cluster = clusters[i]
        nuevo_cluster.extend(clusters[j])
        clusters.append(nuevo_cluster)

        # agrego la unión que realizo
        uniones.append([nombres_de_clusters[i], nombres_de_clusters[j], d, len(nuevo_cluster)])

        # agrego la composición del nuevo cluster
        nombres_de_clusters.append(prox_cluster)
        prox_cluster+=1

        # Actualizar la matriz de distancias
        distancias = actualizar_distancias(distancias, i, j, criterio=criterio_distancia)

        # Sacar la info de los clusters sin fusionar
        clusters = [c for idx,c in enumerate(clusters) if idx not in [i,j]]
        nombres_de_clusters = [n for idx,n in enumerate(nombres_de_clusters) if idx not in [i,j]]


    return clusters, np.array(uniones)


5. Crear un dendograma con el dataset de Iris y verificar si la clusterización generada es o no consistente con lo expresado en los targets

    ```python
        # Cargamos el dataset que usaremos hoy
        from sklearn.datasets import load_iris
        iris_dataset = load_iris()

        X = pd.DataFrame(iris_dataset.data)
        y = pd.Series(iris_dataset.target)
    ```

    Explorar las siguientes condiciones:

    1. Criterio de distancias

    1. ¿Se obtiene una mejor, peor o similar clusterización al normalizar los atributos?
 

##  EM para GMMs

### Inicialización:
Decidir cuantos clusters (c) utilizaremos. Luego, inicializar los parámetros para $c$ normales: $c$ medias ($\mu_c$), $c$  desvios ($\sigma_c$), y $c$ pesos ($\pi_c$).

### Paso E:

Calcular para cada punto $x^{(i)}$ la probabilidad ($r_{ic}$) de que el punto pertenezca al cluster c:
$$r_{ic} = P(c | x^{(i)}) = \frac{P(c)P(x^{(i)} | c)}{P(x^{(i)})} = \frac{\pi_c N(x^{(i)} \ | \ \mu_c,\sigma_c)}{\Sigma_{k=1}^K \pi_k N(x^{(i)} \ | \ \mu_k,\sigma_k)}$$

donde 

$${\displaystyle {\begin{aligned}N(x^{(i)},\mu_c,\sigma_c)&{}={\frac {1}{\sigma_c {\sqrt {2\pi }}}}e^{-{\frac {(x^{(i)}-\mu_c )^{2}}{2\sigma_c ^{2}}}},\quad \pi = 3.141592... .\\\end{aligned}}}$$

### Paso M:
Para cada cluster c, actualizar $\pi_c$, $\mu_c$, y $\sigma_c$ según los datos:

$$ N_c = \sum_{i}{r_{ic}}$$

$$ \mu^{(nuevo)}_c = \frac{1}{N_c}\sum_{i}{r_{ic}x^{(i)}}$$
$$ \sigma^{(nuevo)}_c = \frac{1}{N_c}\sum_{i}{r_{ic}(x^{(i)} - \mu^{(nuevo)}_c)^2} $$
$$ \pi^{(nuevo)}_c = \frac{N_c}{n}$$



### Iterar 
Iterar hasta que la log-likelihood del modelo converja:

$$ln \ p(\boldsymbol{X} \ | \ \boldsymbol{\pi},\boldsymbol{\mu},\boldsymbol{\sigma}) \ = \ \Sigma_{i=1}^N \ ln(\Sigma_{k=1}^K \pi_k N(x^{(i)} \ | \ \mu_k,\sigma_k))$$

In [ ]:
%matplotlib inline
import scipy.stats
from sklearn.cluster import KMeans
import math 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

def gmm_train(X, n_clusters, tolerance = 1e-3):
    v_mu, v_sigma, v_pi = inicializacion(n_clusters, X)
    old_llk = 0
    tolerance = 1e-3 # indica la mínima ganancia en llk para que el algoritmo continúe.
    gain = np.inf

    while(gain > tolerance):
        print_iter(v_mu, v_sigma, v_pi, n_clusters)
        r = e(X, n_clusters, v_mu, v_sigma, v_pi)
        v_mu, v_sigma, v_pi = m(X, n_clusters, r)
        llk = log_likelihood(X, v_mu, v_sigma, v_pi, n_clusters)
        print("Log likelihood: ", llk)
        gain = llk - old_llk
   
    return v_mu, v_sigma, v_pi

def inicializacion(n_clusters, X):
    ##################
    # COMPLETAR
    # Inicializar v_mu, v_sigma, v_pi (vectores que contienen una componente por cluster)
    ##################
    
    return v_mu, v_sigma, v_pi

def norm(x, mu, sigma):
    return scipy.stats.norm(mu, sigma).pdf(x)

def e(X, n_clusters, v_mu, v_sigma, v_pi):
    r = np.ones((len(X), n_clusters))
    
    ##################
    # COMPLETAR
    # r[i, c] debera contener la probabilidad estimada P(c | x^(i))
    ##################
    
    return r

def m(X, n_clusters, r):
    N = np.zeros(n_clusters)
    v_mu_new = np.zeros(n_clusters)
    v_sigma_new = np.zeros(n_clusters)
    v_pi_new = np.zeros(n_clusters)
    
    ##################
    # COMPLETAR
    # v_mu_new, v_sigma_new, v_pi_new deberán contener los valores actualizados.
    ##################
    
    return v_mu_new, v_sigma_new, v_pi_new


def log_likelihood(X, v_mu, v_sigma, v_pi, n_clusters):
    suma = 0
    for i in range(len(X)):
        suma_k = 0
        for k in range(n_clusters):
            suma_k += v_pi[k] * norm(X[i], v_mu[k], v_sigma[k])
        suma += math.log(suma_k)
    return suma


def print_iter( v_mu, v_sigma, v_pi, n_clusters):
    for c in range(n_clusters):
        print("C{} => mu={} +- {} (pi={})".format(c+1, "%.2f" % v_mu[c], "%.2f" % v_sigma[c], "%.2f" % v_pi[c]), end=" | ")
    print()

def dibujar(X, v_mu, v_sigma, v_pi, n_clusters, title="GMMs"):
    plt.figure()
    plt.title(title)
    colors = sns.color_palette("hls", n_clusters)
    plt.plot(X, [1] * len(X), ".")
    for k in range(n_clusters):
        col = colors[k]
#         plt.plot([v_mu[k], v_mu[k]], [0, 2], label="$\mu_{}$".format(k), color=col)
        plt.plot([v_mu[k] - v_sigma[k], v_mu[k] + v_sigma[k]], [1.1,1.1], color=col, label="$\mu_{}$".format(k))
        plt.plot([v_mu[k] - v_sigma[k], v_mu[k] - v_sigma[k]], [0.9,1.2], color=col)
        plt.plot([v_mu[k] + v_sigma[k], v_mu[k] + v_sigma[k]], [0.9,1.2], color=col)
    plt.yticks([])
    plt.ylim([0, 2])
    plt.legend()


In [ ]:
# Datos para ejemplo
np.random.seed(134)
X = np.array([10, 11, 12, 20, 21, 22, 30, 31, 32, 29, 33])
n_clusters = 3

# DESCOMENTAR
v_mu, v_sigma, v_pi = gmm_train(X, n_clusters)
dibujar(X, v_mu, v_sigma, v_pi, n_clusters, title="implementacion propia")
#print("DESCOMENTAR lo anterior")

print("Salida esperada:")
# SALIDA ESPERADA:
sklearn_gmm = GaussianMixture(n_components=3)
sklearn_gmm.fit(X.reshape(-1, 1))

dibujar(X, sklearn_gmm.means_.ravel(), sklearn_gmm.covariances_.ravel(), sklearn_gmm.weights_.ravel(), n_clusters, title="sklearn")


-------------------------------------------------------------------